In [18]:
#from utils import *

import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import re
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sub-task A - Offensive language identification
Given a text, your system should categorise it into OFF (offensive) and NOT (not offensive).

## Load Data

In [6]:
path = '../data/Test A Release/testset-taska.tsv'
path = '../data/start-kit/training-v1/offenseval-training-v1.tsv'
data_train = pd.read_table(path,index_col='id')
data_train.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


## Data Transformation and Cleaning

Work on the text of each tweet : Extracts : 
- Hashtags
- Words
- Words cut on CAPs and Numbers 

In [16]:

data_train['hashtags'] = data_train['tweet'].apply(lambda x : re.compile('#(\w+)').findall(x))
data_train['words'] = data_train['tweet'].apply(lambda x : re.compile('\w+').findall(x))
data_train['words_cut'] = data_train['tweet'].apply(lambda x : re.compile('[0-9]+|[A-Z][a-z]+|[A-Z]+|[a-z]+').findall(x))
data_train['words_cut_low'] = data_train['words_cut'].apply(lambda l : list(map(lambda x: x.lower(),l)))
data_train['words_clean'] = data_train['words_cut_low'].apply(lambda l : list(filter(lambda x : x != 'user' ,l)))
data_train.head(100)


,tweet,subtask_a,subtask_b,subtask_c,words,words_cut,words_cut_low,words_clean,hashtags
id,,,,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,"[USER, She, should, ask, a, few, native, Ameri...","[USER, She, should, ask, a, few, native, Ameri...","[user, she, should, ask, a, few, native, ameri...","[she, should, ask, a, few, native, americans, ...",[]
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,"[USER, USER, Go, home, you, re, drunk, USER, M...","[USER, USER, Go, home, you, re, drunk, USER, M...","[user, user, go, home, you, re, drunk, user, m...","[go, home, you, re, drunk, maga, trump, 2020, ...","[MAGA, Trump2020]"
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,"[Amazon, is, investigating, Chinese, employees...","[Amazon, is, investigating, Chinese, employees...","[amazon, is, investigating, chinese, employees...","[amazon, is, investigating, chinese, employees...","[Amazon, MAGA, KAG, CHINA, TCOT]"
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"[USER, Someone, should, veTaken, this, piece, ...","[USER, Someone, should, ve, Taken, this, piece...","[user, someone, should, ve, taken, this, piece...","[someone, should, ve, taken, this, piece, of, ...",[]
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,"[USER, USER, Obama, wanted, liberals, amp, ill...","[USER, USER, Obama, wanted, liberals, amp, ill...","[user, user, obama, wanted, liberals, amp, ill...","[obama, wanted, liberals, amp, illegals, to, m...",[]
97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH,"[USER, Liberals, are, all, Kookoo]","[USER, Liberals, are, all, Kookoo]","[user, liberals, are, all, kookoo]","[liberals, are, all, kookoo]",[]
77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN,"[USER, USER, Oh, noes, Tough, shit]","[USER, USER, Oh, noes, Tough, shit]","[user, user, oh, noes, tough, shit]","[oh, noes, tough, shit]",[]
52415,@USER was literally just talking about this lo...,OFF,TIN,GRP,"[USER, was, literally, just, talking, about, t...","[USER, was, literally, just, talking, about, t...","[user, was, literally, just, talking, about, t...","[was, literally, just, talking, about, this, l...",[]
45157,@USER Buy more icecream!!!,NOT,NaN,NaN,"[USER, Buy, more, icecream]","[USER, Buy, more, icecream]","[user, buy, more, icecream]","[buy, more, icecream]",[]


In [37]:
data_train.iloc[0].tweet

'@USER She should ask a few native Americans what their take on this is.'

In [173]:
list(map(lambda x: x.lower(),u))

['yolo', 'hehe', 'user']

In [212]:
list(filter(lambda x : x != 's' ,u))

['YOLO', 'USER']

In [50]:
tokenizer = BertTokenizer.from_pretrained('')
tokenizer.tokenize('@USER Sh 🖕🖕🖕🖕🖕 e is drinking WeAreTheChampions Fort so iLoveTheCat much koolaid she’s')

['@',
 'user',
 'sh',
 '[UNK]',
 'e',
 'is',
 'drinking',
 'wear',
 '##eth',
 '##ech',
 '##amp',
 '##ions',
 'fort',
 'so',
 'il',
 '##ove',
 '##the',
 '##cat',
 'much',
 'ko',
 '##ola',
 '##id',
 'she',
 '’',
 's']